In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d, distance

In [2]:
def moment(points, a1, a2, a3):
    result = 0
    for x,y,z in points:
        result += x**a1 * y**a2 * z **a3
    return result

def moment_invariants(points):
    vol = moment(points, 0, 0, 0)
    temp200 =  moment(points, 2, 0, 0)
    temp020 =  moment(points, 0, 2, 0)
    temp002 =  moment(points, 0, 0, 2)
    temp110 =  moment(points, 1, 1, 0)
    temp101 =  moment(points, 1, 0, 1)
    temp011 =  moment(points, 0, 1, 1)
    non_normalized_moment1 = temp200 + temp020 + temp002
    non_normalized_moment2 = temp200 * temp020 +  temp200*temp002 +  temp020*temp002 -temp110**2 -temp101**2 -temp011**2
    non_normalized_moment3 =  temp200 * temp020* temp002 + 2* temp110 * temp101 * temp011 - temp200*temp011**2 -temp020*temp101**2 -temp002 * temp110**2
    result = [(3* vol ** (5/3))/non_normalized_moment1,
              (3* vol ** (10/3))/non_normalized_moment2,
              (vol ** 5)/non_normalized_moment3]
    return result


In [9]:
def get_points_from_matrix(temp,condition=1, dim_count=3):
    temp_indice_specific = np.where(temp==condition)
    points = []
    if len(temp_indice_specific[0])>0:
        tempsize = temp_indice_specific[0].shape[0]
        if len(temp.shape)==3:
            a = temp_indice_specific[0].reshape(tempsize, 1)
            b = temp_indice_specific[1].reshape(tempsize, 1)
            c =  temp_indice_specific[2].reshape(tempsize, 1)
            points = np.concatenate((a, b, c), axis=1)

        elif dim_count==3:
            a = temp_indice_specific[1].reshape(tempsize, 1)
            b = temp_indice_specific[2].reshape(tempsize, 1)
            c = temp_indice_specific[3].reshape(tempsize, 1)
            points = np.concatenate((a, b, c), axis=1)

        elif dim_count==4:
            a1 = temp_indice_specific[0].reshape(tempsize, 1)
            a = temp_indice_specific[1].reshape(tempsize, 1)
            b = temp_indice_specific[2].reshape(tempsize, 1)
            c = temp_indice_specific[3].reshape(tempsize, 1)
            points = np.concatenate((a1, a, b, c), axis=1)

    return points

In [7]:
def create_shape(center, radius, dimension_mat, shape, filled_val=1):
    for x in range(center[0] - radius[0], center[0] + radius[0] + 1):
      for y in range(center[1] - radius[1], center[1] + radius[1] + 1):
          for z in range(center[2] - radius[2], center[2] + radius[2] + 1):
              if (0<= x and x<= dimension_mat.shape[0]-1) and  (0<= y and y<= dimension_mat.shape[1]-1) and  (0<= z and z<= dimension_mat.shape[2]-1):

                  if shape == 'sphere'  and dimension_mat[x, y, z] == 0 and round(distance.euclidean([x, y, z], center)) <= radius[0]:
                    dimension_mat[x, y, z] = filled_val
                  elif shape == 'ellipsie' and dimension_mat[x, y, z] == 0:
                    a = ((x - center[0]) ** 2) / (radius[0] ** 2)
                    b = ((y - center[1]) ** 2) / (radius[1] ** 2)
                    c = ((z - center[2]) ** 2) / (radius[2] ** 2)
                    if round(a+b+c) <= 1:
                      dimension_mat[x, y, z] = filled_val
    return dimension_mat

In [21]:

dimension_mat = np.zeros((15,15,15))
temp_sphere = create_shape([5,6,5], [4,4,4], dimension_mat, 'sphere', filled_val=1)
points = get_points_from_matrix(temp_sphere, 1)
print(moment_invariants(points))


[1.6261601957528915, 11.265715483256985, 99.76867632872049]
